# LSSTCam pointing errors
## Looking at offsets more days.

Craig Lage - 11-Jul-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
eventMaker = TMAEventMaker()

## Running this at the summit, because USDF is still not working well

In [ ]:
startDay = 20250629
endDay = 20250713

els = []
azs = []
ras = []
decs = []
deltaRas = []
deltaDecs = []

filters = []
pressures = []
temps = []
hums = []
times = []
dayObs = startDay
expIds = []
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            cWcs = calExp.getWcs()
            if cWcs == None:
                #print(f"{record.id} had no cWcs.")
                continue
            rWcs = rawExp.getWcs()
            rawSkyCenter = rWcs.getSkyOrigin()
            calExpSkyCenter = cWcs.pixelToSky(rWcs.getPixelOrigin())
            ra = calExpSkyCenter.getRa().asDegrees()
            dec = calExpSkyCenter.getDec().asDegrees()
            expIds.append(record.id)
            els.append(md['ELSTART'])
            azs.append(md['AZSTART'])
            filters.append(md['FILTBAND'])
            pressures.append(md['PRESSURE'])
            temps.append(md['AIRTEMP'])
            hums.append(md['HUMIDITY'])
            times.append((md['MJD-BEG'] + md['MJD-END']) / 2.0)
            ras.append(ra)
            decs.append(dec)
            #print(ra, dec)
            deltaRa = rawSkyCenter.getRa().asArcseconds() - calExpSkyCenter.getRa().asArcseconds()
            deltaDec = rawSkyCenter.getDec().asArcseconds() - calExpSkyCenter.getDec().asArcseconds()
            deltaRas.append(deltaRa)
            deltaDecs.append(deltaDec)
            #print(record.id, deltaRa, deltaDec)
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(ras))
    dayObs = calcNextDay(dayObs)

filename = "/home/c/cslage/u/MTMount/mount_plots/pointing_results_14jul25.pkl"
with open(filename, 'wb') as f:
    pkl.dump([expIds, els, azs, ras, decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters], f)


In [ ]:
filename = "/home/c/cslage/u/MTMount/mount_plots/pointing_results_14jul25.pkl"
with open(filename, 'rb') as f:
    [expIds, els, azs, ras, decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)
len(els)

In [ ]:
startDay = 20250629
int(startDay * 1E5)

In [ ]:
startDay = 20250629
endDay = 20250713
dayObs = startDay

while dayObs <= endDay:
    separation = []
    for i in range(len(ras)):
        if (expIds[i] > dayObs * 1E5) and (expIds[i] < calcNextDay(dayObs) * 1E5):
            sep = np.sqrt(((deltaRas[i] * np.cos(decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
            separation.append(sep)

    print(dayObs, len(separation), np.median(separation))
    dayObs = calcNextDay(dayObs)


In [ ]:
change = 20250708 * 1E5

separation1 = []
separation2 = []
for i in range(len(ras)):
    if (expIds[i] > change):
        sep = np.sqrt(((deltaRas[i] * np.cos(decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
        separation2.append(sep)
    else:
        sep = np.sqrt(((deltaRas[i] * np.cos(decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
        separation1.append(sep)
plt.hist(separation1, bins = 100, range=(0,100), alpha=0.5, label="Before 20250708")
plt.hist(separation2, bins = 100, range=(0,100), alpha=0.5, label="After 20250708")
plt.xlim(0, 100)
plt.legend()
plt.title(f"On sky pointing model error, {startDay}-{endDay}")
plt.xlabel("Error (arseconds)")
plt.savefig("/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_20250629-20250713.png")

In [ ]:
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

In [ ]:
deltaAzs = []
deltaEls = []
good_azs = []
good_els = []
true_azs = []
true_els = []
bad_azs = []
bad_els = []
for index in range(len(ras)):
    if (expIds[index] < change):
        bad_azs.append(azs[index])
        bad_els.append(els[index])
        continue
    skyLocation = SkyCoord(ras[index]*u.deg, decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    true_az = Angle(obsAltAz.az.deg * u.deg)
    wrapped_true_az = true_az.wrap_at(180.0 * u.deg)
    true_azs.append(wrapped_true_az.deg)
    az = Angle(azs[index] * u.deg)
    wrapped_az = az.wrap_at(180.0 * u.deg)
    deltaAz = az.deg - true_az.deg

    if deltaAz > 360.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0

    deltaAz *= 3600.0 * np.cos(obsAltAz.alt.rad)
    deltaEl = (els[index] - obsAltAz.alt.deg) * 3600.0
    true_els.append(obsAltAz.alt.deg)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    good_azs.append(wrapped_az.deg)
    good_els.append(els[index])
    
len(good_els)        

In [ ]:
from scipy.optimize import minimize
def FOM(params, args):
    fom = 0.0
    [good_azs, deltaEls] = args
    [amp, off] = params
    for i in range(len(good_azs)):
        if abs(deltaEls[i]) > 450.0:
            continue
        model = off + amp * np.sin((good_azs[i]) * np.pi / 180.0)
        fom += np.square(model - deltaEls[i])
    return fom

args = [good_azs, deltaEls]
x0 = [400.0, 0.0]
result = minimize(FOM, x0, args=args, method='Powell')
result

In [ ]:
from scipy.optimize import minimize
def FOM(params, args):
    fom = 0.0
    [good_azs, deltaAzs] = args
    [amp, off] = params
    for i in range(len(good_azs)):
        if abs(deltaAzs[i]) > 450.0:
            continue
        model = off + amp * np.cos((good_azs[i]) * np.pi / 180.0)
        fom += np.square(model - deltaAzs[i])
    return fom

args2 = [good_azs, deltaAzs]
x0 = [400.0, 0.0]
result2 = minimize(FOM, x0, args=args2, method='Powell')
result2

In [ ]:
[amp, off] = result.x
[amp2, off2] = result2.x
xs = np.linspace(-180.0, 180.0, 200)
ys = off + amp * np.sin((xs) * np.pi / 180.0)
ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)

plt.figure(figsize=(8,8))
plt.suptitle("LSST Delta AltAz 2025-07-08 - 2025-07-13")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(good_els, deltaEls, c=good_azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(good_azs, deltaEls,c=good_els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.plot(xs, ys, ls = '--', color='black')
plt.text(-100,250,f"deltaEl={off:.1f}+\n{amp:.1f}*sin(az)")
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(good_els, deltaAzs, c=good_azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(good_azs, deltaAzs,c=good_els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys2, ls = '--', color='black')
plt.text(-100,-100,f"deltaAz={off2:.1f}+\n{amp2:.1f}*cos(az)")
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(-180, 180)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.savefig("/home/c/cslage/u/MTMount/mount_plots/Delta_AltAz_14Jul25.png")


In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(good_azs) * np.pi / 180.0, 90.0 - np.array(good_els), marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)
plt.savefig("/home/c/cslage/u/LSSTCam/data/Sky_Coverage_2025-07-08_2025-07-13.png")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(bad_azs) * np.pi / 180.0, 90.0 - np.array(bad_els), marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)
plt.savefig("/home/c/cslage/u/LSSTCam/data/Sky_Coverage_Bad_2025-07-08_2025-07-13.png")

# Generate a tpoint input file with these errors

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_14Jul25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,July 14, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

for i in range(len(good_els)):
    outfile.write(f"{true_azs[i]:.9f}\t{true_els[i]:.6f}\t{good_azs[i]:.9f}\t{good_els[i]:.9f}\n")
outfile.write("END\n")
outfile.close()



In [ ]:
for i in range(len(good_azs)):
    print(good_azs[i], true_azs[i])